# Web-scraping for blogs written in Korean
* documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
* https://scraperwiki.com/2011/12/how-to-scrape-and-parse-wikipedia/

#### Required libraries

In [1]:
import json
import os
import glob
from urllib.request import urlopen
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote
from urllib.parse import urlsplit, parse_qs
import lxml.etree
import urllib
import urllib.parse
import re

# Step 1: Scrape table on Wikipedia (using Beautiful Soup)
* Not recommended as Wiki already provides dump using its own API.

In [2]:
movie_list = ['가을동화','별에서_온_그대','그들이_사는_세상','하얀거탑_(2007년_드라마)','찬란한_유산','아내의_유혹', '도깨비_(드라마)','굿_와이프_(2016년_드라마)','비밀의_숲','아내의_자격', 
              '프로듀사','상속자들','연애의_발견','최고의_한방','푸른_바다의_전설','베이비시터_(드라마)','밀회','커피프린스_1호점','로열_패밀리_(드라마)','연애시대_(드라마)',
             '응답하라_1988','풍문으로_들었소','자이언트_(드라마)','미생_(드라마)','아이가_다섯','38사기동대','시그널_(드라마)','최고의_사랑','온에어_(드라마)','마이_프린세스','넝쿨째_굴러온_당신',
             '아르곤_(2003년_드라마)','옥탑방_고양이','품위있는_그녀','구해줘','뿌리깊은_나무_(2011년_드라마)','펀치_(드라마)','혼술남녀','치즈인더트랩_(드라마)',
             '청춘시대_(드라마)','운명처럼_널_사랑해','끝에서_두번째_사랑','판타스틱_(드라마)','구르미_그린_달빛_(드라마)','9회말_2아웃',
             '허준_(드라마)','기억_(드라마)','쌈_마이웨이','수상한_파트너','힘쎈여자_도봉순','태양의_여자','시크릿_가든_(드라마)','내_남자의_여자','황금의_제국',
             '지붕뚫고_하이킥!','신의_저울','그_겨울,_바람이_분다','아버지가_이상해','학교_2017','역적_:_백성을_훔친_도적',
             '사임당,_빛의_일기','대장금','이웃집_웬수','고백부부','명불허전_(드라마)','당신이_잠든_사이에_(드라마)','마의','태양의_후예','내조의_여왕','반짝반짝_빛나는','마녀보감',
             '결혼계약','골든타임_(드라마)','직장의_신','너의_목소리가_들려','욱씨남정기','해를_품은_달_(드라마)','바람의_화원_(드라마)','질투의_화신','추노','올인','또!_오해영','신사의_품격',
             '낭만닥터_김사부','애인있어요','가족끼리_왜_이래','닥터스_(드라마)','옥중화','오_나의_귀신님','W_(드라마)']

# movie_list = ['가을동화','별에서_온_그대','그들이_사는_세상','하얀거탑_(2007년_드라마)','찬란한_유산','아내의_유혹', '도깨비_(드라마)','굿_와이프_(2016년_드라마)','비밀의_숲','아내의_자격', 
#               '프로듀사','상속자들','연애의_발견','최고의_한방','푸른_바다의_전설','베이비시터_(드라마)','밀회','커피프린스_1호점','로열_패밀리_(드라마)','연애시대_(드라마)']

In [3]:
def wiki_url(movie_title):
    movie_title_urls = set()
    for title in movie_title:
        url = "https://ko.wikipedia.org/wiki/" + title
        movie_title_urls.add(url)
    return(list(movie_title_urls)) # Convert set to list

In [15]:
def get_table(url):
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        table_box = soup.find("table", {"class": "infobox"})
        info_table = {}
        for row in table_box.findAll("tr"):
            for cl in row.findAll("th"): # Table column
                variable = cl.find(text = True)
                for x in row.findAll("td"): # Table cell value
                    value = x.find(text = True)
                    info_table[variable] = value
        return info_table 
    except:
        pass


In [16]:
def wiki_table_summary(movie_lists):
    fail_list = []
    movie_dict = {}
    movie_urls = wiki_url(movie_lists)

    for url in movie_urls:
        title = urlsplit(url).path.split('/')[2]
        movie_info = get_table(url)
        if movie_info is None:
            fail_list.append(title)
        else:
            movie_dict[title] = movie_info
            
    print(fail_list)
    return movie_dict

movie_wiki = wiki_table_summary(movie_list)
#movie_wiki

[]


In [450]:
#movie_wiki['별에서_온_그대']

# Step 2: Scrape data on Wikipedia (using Mediwiki API)
* Better quality data than BeautifulSoup

In [558]:
movie_list = ['가을동화','별에서_온_그대','그들이_사는_세상']

# Later:
# * Need to sort by Korean characters

In [3]:
infobox_fields = ['방송명','원제','시청등급','장르','방송 시간','방송 분량','방송 기간','방송 횟수','방송 국가','방송 채널','기획','제작','총감독','감독','연출','조연출',
          '원작','각본','총괄프로듀서','프로듀서','출연자','분장','미용','의상디자인','의상','음성','자막','데이터 방송','여는 곡','닫는 곡','촬영감독','촬영보',
         '조명감독','조명1st','조명팀','미술감독','미술개발','미술행정','극본','대본','구성','출연자','제작사','자막','주요 촬영지']

text_fields = ["== 줄거리 ==","== 등장 인물 ==","== OST ==","== 수상 목록 ==","== 역사적 사실과 오류 ==","== 이모저모 ==","== 시청률 ==","== 사운드트랙 ==","== 표절 논란 ==","== 결방 사유 및 연장 ==","== 수상 경력 ==","== 동시간대 드라마 ==",
              "== 편성 변경 ==","== 경쟁작 ==","== 한류 붐 ==","== 같이 보기 ==","== 오리지널 사운드트랙 ==","== 수상 경력 및 후보 ==","== 관련 도서 ==","== 제작진 ==","== 연장 ==","== 해외 방영 ==","== 특이 사항 ==","== 음악 ==",
               "== 촬영 장소 ==","== 기획의도 ==","== 회차별 줄거리 ==","== 드라마 속 드라마 ==","== 관련 서적 ==","== 원작 ==","== 주제 ==","== 제작 ==","== 인기 ==","== 수상 경력 및 후보 =="]

# Later:
# * Need to sort by Korean characters

In [4]:
def get_raw_text(title):
    params = {"format":"xml", "action":"query", "prop":"revisions", "rvprop":"timestamp|user|comment|content"}
    params["titles"] = "API|%s" % urllib.parse.quote(title.encode("utf8"))
    qs = "&".join("%s=%s" % (k, v)  for k, v in params.items()) #query string
    url = "http://ko.wikipedia.org/w/api.php?%s" % qs
    tree = lxml.etree.parse(urlopen(url))
    revs = tree.xpath('//rev')
    raw_text = revs[-1].text
    return raw_text

In [5]:
def extract_infobox(raw_text, infobox_fields):
    """
    Using Wiki's unique format rules, extract infobox data
    Returns infobox data in a dictionary
    """
    infobox_start_idx = raw_text.find("{{텔레비전 프로그램 정보")
    infobox_end_idx = raw_text.find("《'''")
    infobox_text = raw_text[infobox_start_idx:infobox_end_idx]
    
    infobox_field_pos = [infobox_text.find(field) for field in infobox_fields]
    field_pos_pair = list(zip(infobox_field_pos,infobox_fields))
    field_pos_pair = sorted(field_pos_pair, key=lambda tup: tup[0]) 
    
    infobox_dic = {}
    for idx, field in enumerate(field_pos_pair):
        if field[0] <0:  # If there is no value, pass
            pass
        else: 
            field_name = field[1]
            start_idx = field[0] + len(field_name) + len(" = ")
            if idx == len(field_pos_pair) - 1:
                end_idx = len(infobox_text)
            else: 
                end_idx = field_pos_pair[idx + 1][0]-1
                
            infobox_dic[field_name] = infobox_text[start_idx:end_idx].rstrip() 
    return infobox_dic

In [6]:
#test_dict = extract_infobox(text_raw,infobox_fields)
#print(test_dict['연출'])

In [7]:
def remove_ref(input_string):
    ref_pos = []
    back_ref_pos = []
    for idx, char in enumerate(input_string):
        test_string_1 = input_string[idx:idx+5]
        if test_string_1 == "<ref>":
            ref_pos.append(idx)
    for idx, char in enumerate(input_string):
        test_string_2 = input_string[idx:idx+6]
        if test_string_2 == "</ref>":
            back_ref_pos.append(idx)
            
    ref_pair = list(zip(ref_pos,back_ref_pos))
    remove_text = []
    for pair in ref_pair:
        target_text = input_string[pair[0]:pair[1]+6]
        remove_text.append(target_text)
    
    clean_input_string = input_string
    for text in remove_text:
        clean_input_string = clean_input_string.replace(text, "")
        
    return clean_input_string    

In [8]:
def extract_summary(raw_text):
    """
    Using Wiki's unique format rules, extract summary
    Returns summary as string
    """
    summary_start_idx = raw_text.find("《'''")
    summary_end_idx = raw_text.find("==")
    summary_text = raw_text[summary_start_idx:summary_end_idx].rstrip()
    summary_text_clean = remove_ref(summary_text)
    return summary_text_clean

In [9]:
#test_summary = extract_summary(text_raw)
#print(test_summary)

In [10]:
def extract_tag(raw_text):
    """
    Using Wiki's unique format rules, extract tags
    Returns tags as list
    """
    tag_start_idx = raw_text.find("[[분류:")
    tag_end_idx = len(raw_text)
    tag_text = raw_text[tag_start_idx:tag_end_idx]
    cat_pos = [pos for pos, char in enumerate(tag_text) if char == ":"]
    tags = []
    for idx, pos in enumerate(cat_pos):
        start_idx = pos+1
        if idx == len(cat_pos)-1:
            end_idx = len(tag_text)
        else:
            end_idx = cat_pos[idx + 1] - 7
        tag = tag_text[start_idx:end_idx]
        tags.append(tag)
    return tags

# Step 2: Clean up the data (In the text section)
* parse
* date format
* other dictionaies & indecies
* scrape plot
* create English version

## 등장 인물

In [11]:
#test_people = (wiki_metadata['별에서_온_그대']['등장 인물'])

In [12]:
def find_pos_string(input_string,find_string):
    target_pos = []
    for idx, char in enumerate(input_string):
        test_string = input_string[idx:idx+3]
        if test_string == find_string:
            target_pos.append(idx)
    return target_pos    

In [22]:
def cast_category(cast_raw_text):
    """
    Using Wiki's unique format, extract cast category chunks.
    Return dic with cast category as key, and its text(raw) as value.
    """
    category_divider_idx = []
    cast_category = []
    cast_dic = {}
    category_divider_pos = find_pos_string(cast_raw_text, "===")
    for x,y in zip(category_divider_pos[0::2], category_divider_pos[1::2]):
        idx = list((x,y))
        category_divider_idx.append(idx)
    for idx, pair in enumerate(category_divider_idx):
        title_start_idx = pair[0]+4
        title_end_idx = pair[1]-1
        text_start_idx = pair[0]+14
        if idx == len(category_divider_idx)-1:
            text_end_idx = len(cast_raw_text)
        else:
            text_end_idx = category_divider_idx[idx+1][0]-2
        
        cast_dic[cast_raw_text[title_start_idx:title_end_idx]]= cast_name(cast_raw_text[text_start_idx:text_end_idx])  #cast_raw_text[text_start_idx:text_end_idx]
    return cast_dic

In [23]:
#cast_dict = cast_category(test_people)
#print(cast_dict['그 외 인물'])

In [24]:
def cast_name(cast_dic):
    cast_text = cast_dic
    person_pos = [pos for pos, char in enumerate(cast_text) if char == "*"]
    name_stop = [pos for pos, char in enumerate(cast_text) if char == "]"]
    cast_names = []
    for pos in person_pos:
        name_field = cast_text[pos:pos+25]
        end_idx = name_field.find(']]') + pos
        name = cast_text[pos+4:end_idx]
        cast_names.append(name)
    return cast_names
    
#cast_names = cast_name(test_people)
#cast_names

## 시청율

In [25]:
# Later

## OST

In [26]:
# Later 

## Combine text extraction functions

In [27]:
def extract_text(raw_text):
    """
    Using Wiki's unique format rules, extract text
    Returns text in dictionary
    """
    text_start_idx = raw_text.find("==")
    text_end_idx = raw_text.find("[[분류:")
    text = raw_text[text_start_idx:text_end_idx]
    
    text_pos = [text.find(field) for field in text_fields]
    text_pos_pair = list(zip(text_pos, text_fields))
    text_pos_pair = sorted(text_pos_pair, key = lambda tup:tup[0])
    
    text_dic = {}
    for idx, field in enumerate(text_pos_pair):
        if field[0] <0:  # If there is no value, pass
            pass
        else: 
            field_name = field[1][3:-3]
            start_idx = field[0] + len(field_name) + 6
            if idx == len(text_pos_pair) - 1:
                end_idx = len(text)
            else: 
                end_idx = text_pos_pair[idx + 1][0]-1

            """
            Handle cast starts
            """
            
            if field_name == "등장 인물":
                text_dic[field_name] = {}
                cast_raw_text = remove_ref(text[start_idx:end_idx].rstrip())
                cast_dict = cast_category(cast_raw_text)
                
                text_dic[field_name] = cast_dict
            else:           
                text_dic[field_name] = remove_ref(text[start_idx:end_idx].rstrip())
    return text_dic

In [28]:
# text_dic = extract_text(text_raw)
# text_dic.keys()
# print(text_dic['이모저모'])

# Step 3: Scrape everything

In [29]:
def extract_everything(title_lists):
    wiki_dic = {}
    for title in title_lists:
        wiki_dic[title] = {}
        raw_text = get_raw_text(title)
        
        infobox_dic = extract_infobox(raw_text, infobox_fields)
        summary = extract_summary(raw_text)
        tags = extract_tag(raw_text)
        texts = extract_text(raw_text)
        
        wiki_dic[title] = infobox_dic
        wiki_dic[title]['summary'] = summary
        wiki_dic[title]['tags'] = tags
        wiki_dic[title] = texts
    return wiki_dic

In [30]:
wiki_metadata = extract_everything(movie_list)
#print(wiki_metadata['별에서_온_그대']['등장 인물']['도민준 주변 인물'])

In [31]:
wiki_metadata['별에서_온_그대'].keys()

dict_keys(['사운드트랙', '결방 사유 및 연장', '등장 인물', '표절 논란', '시청률', '수상 경력', '이모저모', '동시간대 드라마'])

In [34]:
wiki_metadata['별에서_온_그대']['등장 인물']['특별 출연']

['유인영',
 '유준상 (배우)|유준상',
 '달샤벳',
 '김생민',
 '박정아 (1981년)|박정아',
 '장항준',
 '손은서',
 '전인택',
 '김수로 (배우)|김수로',
 '정은표',
 '정윤기 (스타일리스트)|정윤기',
 '박영규 (배우)|박영규',
 '장동규',
 '조영구',
 '노형욱 (배우)|노형욱',
 '수지 (대한민국의 가수)|수지',
 '연우진',
 '류승룡',
 '양주호',
 '김성준 (기자)|김성준',
 '박산다라']

In [37]:
# Convert dictionary into json
import json

In [55]:
wiki_meta_json = json.dumps(wiki_metadata, ensure_ascii=False)
loaded_wiki_meta_json = json.loads(wiki_meta_json)

In [ ]:
loaded_wiki_meta_json[]

# Step 3: Save in database

In [57]:
import elasticsearch
import os

In [58]:
es_cluster = 'http://localhost:9200/'
es_index = 'k5_test'
es_type = 'doc'

In [59]:
es = elasticsearch.Elasticsearch(es_cluster)

In [88]:
idx = 0
movie_titles = wiki_metadata.keys()

for title in movie_titles:
    es.index(index = 'k5_movie', doc_type = 'doc', id = idx, body = {
        'title' : title,
        #'cast' : wiki_metadata[title]['등장 인물'],
        #'plot' : wiki_metadata[title]['줄거리'],
        'body' : wiki_metadata[title]
    })
    idx += 1

In [94]:
es.get(index = 'k5_movie', doc_type = 'doc', id = 1)

{'_id': '1',
 '_index': 'k5_movie',
 '_source': {'body': {'등장 인물': {'특별 출연': ['대한제국 순종|순종', '', '주상욱', '2011년']},
   '시청률': '\n{|class=wikitable\n!colspan="6" | [[2011년]]\n|-align=center\n!rowspan="2"|회차\n!rowspan="2"|방송일\n!colspan="2"|[[TNmS]] 시청률\n!colspan="2"|[[AGB 닐슨 미디어 리서치|AGB]] 시청률\n|-\n!대한민국(전국)\n!서울(수도권)\n!대한민국(전국)\n!서울(수도권)\'\'\'\n|-align=center\n!제1회\n|[[1월 5일]]\n|13.0%\n|17.1%\n|15.9%\n|17.5%\n|-align=center\n!제2회\n|[[1월 6일]]\n|14.3%\n|17.2%\n|17.6%\n|19.8%\n|-align=center\n!제3회\n|[[1월 12일]]\n|16.2%\n|20.0%\n|20.0%\n|21.7%\n|-align=center\n!제4회\n|[[1월 13일]]\n|<span style=color:red>\'\'\'17.0%\'\'\'</span>\n|<span style=color:red>\'\'\'22.7%\'\'\'</span>\n|<span style=color:red>\'\'\'20.9%\'\'\'</span>\n|<span style=color:red>\'\'\'23.0%\'\'\'</span>\n|-align=center\n!제5회\n|[[1월 19일]]\n|16.9%\n|21.9%\n|18.8%\n|21.0%\n|-align=center\n!제6회\n|[[1월 20일]]\n|14.0%\n|18.4%\n|18.8%\n|21.3%\n|-align=center\n!제7회\n|[[1월 26일]]\n|14.1%\n|18.2%\n|15.7%\n|17.7%\n|-align=center\n!제8회\n|[[1